## Zadania: Model agnostic gateway production


## Zadanie 1 — Routing po nagłówku X-Model

In [ ]:
from dataclasses import dataclass
from typing import Optional, Dict, Any

def openai_gpt(prompt: str) -> str: return f"[openai:gpt] {prompt[:40]}..."
def bedrock_claude(prompt: str) -> str: return f"[bedrock:claude] {prompt[:40]}..."

REGISTRY: Dict[str, Any] = {
    "openai:gpt-5-mini": openai_gpt,
    "aws:claude-3-7": bedrock_claude,
}

@dataclass
class Ctx:
    model: Optional[str]
    prompt: str

def pick_model(ctx: Ctx):
    # TODO: wybierz funkcję z REGISTRY na podstawie ctx.model; wpp. domyślny openai:gpt-5-mini
    return REGISTRY.get(ctx.model or "", REGISTRY["openai:gpt-5-mini"])

fn = pick_model(Ctx("aws:claude-3-7","Hello"))
print(fn("Test"))

## Zadanie 2 — Fallback + log kosztu

In [ ]:
COSTS = {"openai:gpt-5-mini":0.0003,"aws:claude-3-7":0.0005}
def run(ctx: Ctx):
    try:
        fn = pick_model(ctx)
        out = fn(ctx.prompt)
        key = ctx.model if ctx.model in REGISTRY else "openai:gpt-5-mini"
    except Exception:
        out = REGISTRY["openai:gpt-5-mini"](ctx.prompt)  # TODO: fallback
        key = "openai:gpt-5-mini"
    log = {"model": key, "est_cost_per_1k": COSTS.get(key,0.0)}  # TODO: realnie po tokenach
    return out, log
print(run(Ctx("aws:claude-3-7","Krótka odpowiedź o Pythonie.")))

## Zadanie 3 — Human-in-the-Loop (approve/edit/reject)

In [ ]:
from enum import Enum
class Decision(str, Enum):
    approve="approve"; edit="edit"; reject="reject"

def hitl(output: str, decision: Decision, edited: str|None=None) -> str:
    # TODO: jeśli approve->output; edit->edited; reject->komunikat o odrzuceniu
    if decision==Decision.approve: return output
    if decision==Decision.edit and edited: return edited
    if decision==Decision.reject: return "[REJECTED] Do poprawy."
    return output

final = hitl(*run(Ctx("openai:gpt-5-mini","Streszcz Python w 1 zdaniu."))[0:1], decision=Decision.edit)  # przykładowe użycie
print(final)